In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
from sklearn.utils import shuffle

In [3]:
import warnings
warnings.filterwarnings("ignore")

### 1. Load data files as dataframes

I load .idomaar files as dataframes because it is more familiar format for me and it is easier to work with.

In [2]:
people_df = pd.read_csv("ThirtyMusic/entities/persons.idomaar",sep='\t', 
                        names = ['object', 'id', 'ts', 'properties', 'linked_entities'])
people_df.head()

,object,id,ts,properties,linked_entities
0,person,145148,-1,"{""MBID"":null, ""name"":""Everything+Is+Illuminated""}",{}
1,person,297899,-1,"{""MBID"":null, ""name"":""Robin+O%27Brien""}",{}
2,person,250429,-1,"{""MBID"":null, ""name"":""Nicholas+Gunn++(2012)""}",{}
3,person,32765,-1,"{""MBID"":null, ""name"":""Aspasia+Stratigou""}",{}
4,person,18689,-1,"{""MBID"":null, ""name"":""Allison+Veltz""}",{}


In [3]:
people_df.shape

(595049, 5)

In [4]:
tracks_df = pd.read_csv("ThirtyMusic/entities/tracks.idomaar",sep='\t', 
                        names = ['object', 'id', 'ts', 'properties', 'linked_entities'])
tracks_df.head()

,object,id,ts,properties,linked_entities
0,track,0,-1,"{""duration"":-1,""playcount"":4,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":0}],""albums""..."
1,track,1,-1,"{""duration"":-1,""playcount"":495,""MBID"":null,""na...","{""artists"":[{""type"":""person"",""id"":1}],""albums""..."
2,track,2,-1,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":2}],""albums""..."
3,track,3,-1,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":3}],""albums""..."
4,track,4,-1,"{""duration"":-1,""playcount"":1,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":4}],""albums""..."


In [5]:
tracks_df.shape

(5675143, 5)

In [6]:
sessions_df = pd.read_csv("ThirtyMusic/relations/sessions.idomaar",sep='\t', 
                        names = ['object', 'id', 'ts', 'properties', 'linked_entities'])
sessions_df.head()

,object,id,ts,properties,linked_entities
0,event.session,287144,1390231051,"{""numtracks"":23,""playtime"":4547} {""subjects"":[...",NaN
1,event.session,287145,1390241844,"{""numtracks"":11,""playtime"":2907} {""subjects"":[...",NaN
2,event.session,287146,1390303249,"{""numtracks"":16,""playtime"":3191} {""subjects"":[...",NaN
3,event.session,287147,1390481828,"{""numtracks"":5,""playtime"":1162} {""subjects"":[{...",NaN
4,event.session,287140,1421443687,"{""numtracks"":2,""playtime"":250} {""subjects"":[{""...",NaN


In [7]:
sessions_df.shape

(2764474, 5)

In [8]:
#split 'properties' column in the sessions_df
splitted = sessions_df['properties'].str.split(" ", n = 1, expand = True)
sessions_df['properties'] = splitted[0]
sessions_df['linked_entities'] = splitted[1]
sessions_df.head()

,object,id,ts,properties,linked_entities
0,event.session,287144,1390231051,"{""numtracks"":23,""playtime"":4547}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
1,event.session,287145,1390241844,"{""numtracks"":11,""playtime"":2907}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
2,event.session,287146,1390303249,"{""numtracks"":16,""playtime"":3191}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
3,event.session,287147,1390481828,"{""numtracks"":5,""playtime"":1162}","{""subjects"":[{""type"":""user"",""id"":44361}],""obje..."
4,event.session,287140,1421443687,"{""numtracks"":2,""playtime"":250}","{""subjects"":[{""type"":""user"",""id"":42773}],""obje..."


### 2. Form user_id and track_id columns, create new sessions dataframe with these columns

In [9]:
def extract_track_and_user(row):
    '''Extracts user_id and track_id from sessions data'''
    json_lst = json.loads(row['linked_entities'])
    user_id = json_lst['subjects'][0]['id']
    track_ids = [obj['id'] for obj in json_lst['objects']]
    new_rows_dict = {
                    'session_id' : row['id'],
                    'ts': row['ts'],
                    'user_id' : user_id,
                    'track_id' : track_ids 
                   } 
    return pd.DataFrame(new_rows_dict)

In [15]:
%%time
#apply custom function to sessions_df, form new df
new_sessions_df = pd.concat(list(sessions_df.apply(lambda row: extract_track_and_user(row), axis=1)))

CPU times: user 44min 42s, sys: 5min 35s, total: 50min 18s
Wall time: 54min 36s


In [16]:
new_sessions_df

,session_id,ts,user_id,track_id
0,287144,1390231051,44361,4698874
1,287144,1390231051,44361,838286
2,287144,1390231051,44361,2588097
3,287144,1390231051,44361,2746740
4,287144,1390231051,44361,3873988
...,...,...,...,...
0,2480032,1407938059,33058,906373
0,2480033,1407939579,33058,512708
1,2480033,1407939579,33058,2672866
2,2480033,1407939579,33058,1558581


In [17]:
new_sessions_df = new_sessions_df.reset_index(drop=True)

In [18]:
new_sessions_df

,session_id,ts,user_id,track_id
0,287144,1390231051,44361,4698874
1,287144,1390231051,44361,838286
2,287144,1390231051,44361,2588097
3,287144,1390231051,44361,2746740
4,287144,1390231051,44361,3873988
...,...,...,...,...
31351940,2480032,1407938059,33058,906373
31351941,2480033,1407939579,33058,512708
31351942,2480033,1407939579,33058,2672866
31351943,2480033,1407939579,33058,1558581


In [19]:
#save new sessions df to pickle
new_sessions_df.to_pickle('data/new_sessions_df.pickle')

### 3. Form person_id column, create new tracks dataframe with this column

In [20]:
def extract_person(row):
    json_lst = json.loads(row['linked_entities'])
    person_id = json_lst['artists'][0]['id']

    new_rows_dict = {
                    'track_id' : row['id'],
                    'person_id' : person_id,
                   } 
    return new_rows_dict

In [21]:
%%time
new_tracks_df = pd.DataFrame(list(tracks_df.apply(lambda row: extract_person(row), axis=1)))

CPU times: user 2min 22s, sys: 12.3 s, total: 2min 34s
Wall time: 2min 37s


In [22]:
new_tracks_df = new_tracks_df.drop_duplicates()

In [23]:
new_tracks_df

,track_id,person_id
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
5675137,5023104,595139
5675138,5023105,187223
5675140,5023106,595140
5675141,5023107,549669


In [24]:
new_tracks_df.to_pickle('data/new_tracks_df.pickle')

### 4. Merge new sessions and tracks dataframes

In [3]:
new_sessions_df = pd.read_pickle('data/new_sessions_df.pickle')
new_tracks_df = pd.read_pickle('data/new_tracks_df.pickle')

In [4]:
new_sessions_df.shape

(31351945, 4)

In [5]:
new_tracks_df.shape

(4519319, 2)

In [6]:
new_tracks_df['track_id'].value_counts()

1744350    2
976246     2
94083      2
151793     2
915661     2
          ..
3325182    1
3321084    1
3316986    1
3312888    1
0          1
Name: track_id, Length: 4519105, dtype: int64

In [7]:
merged_df = new_sessions_df.merge(new_tracks_df, on=['track_id'], how='inner', copy = False)

In [8]:
merged_df.head()

,session_id,ts,user_id,track_id,person_id
0,287144,1390231051,44361,4698874,142266
1,287144,1390231051,44361,838286,107103
2,982046,1405452797,32894,838286,107103
3,982049,1405702985,32894,838286,107103
4,1873088,1406217037,23183,838286,107103


In [9]:
merged_df.shape

(31376602, 5)

In [10]:
merged_df.to_pickle('data/merged_df.pickle')

### 5. Timestamp to date and time

In [11]:
#%%time
date_time = pd.to_datetime(merged_df['ts'], unit = 's').astype(str).str.split(" ", n = 1, expand = True)

In [12]:
merged_df['date'] = date_time[0]
merged_df['time'] = date_time[1]

In [13]:
merged_df

,session_id,ts,user_id,track_id,person_id,date,time
0,287144,1390231051,44361,4698874,142266,2014-01-20,15:17:31
1,287144,1390231051,44361,838286,107103,2014-01-20,15:17:31
2,982046,1405452797,32894,838286,107103,2014-07-15,19:33:17
3,982049,1405702985,32894,838286,107103,2014-07-18,17:03:05
4,1873088,1406217037,23183,838286,107103,2014-07-24,15:50:37
...,...,...,...,...,...,...,...
31376597,540638,1415960966,24700,2564433,321698,2014-11-14,10:29:26
31376598,540638,1415960966,24700,2602587,325968,2014-11-14,10:29:26
31376599,540638,1415960966,24700,720394,87396,2014-11-14,10:29:26
31376600,540638,1415960966,24700,720401,87396,2014-11-14,10:29:26


In [14]:
merged_df.to_pickle('data/merged_df_datetime.pickle')

### 6. Get artists dictionary (person_id: name)

In [3]:
people_df

,object,id,ts,properties,linked_entities
0,person,145148,-1,"{""MBID"":null, ""name"":""Everything+Is+Illuminated""}",{}
1,person,297899,-1,"{""MBID"":null, ""name"":""Robin+O%27Brien""}",{}
2,person,250429,-1,"{""MBID"":null, ""name"":""Nicholas+Gunn++(2012)""}",{}
3,person,32765,-1,"{""MBID"":null, ""name"":""Aspasia+Stratigou""}",{}
4,person,18689,-1,"{""MBID"":null, ""name"":""Allison+Veltz""}",{}
...,...,...,...,...,...
595044,person,544215,-1,"{""MBID"":null, ""name"":""Sanaa+Kariakoo""}",{}
595045,person,298403,-1,"{""MBID"":null, ""name"":""Rock-a-teens""}",{}
595046,person,450896,-1,"{""MBID"":null, ""name"":""Jennifer+Lopez+Ft.+DJ+Mu...",{}
595047,person,53831,-1,"{""MBID"":null, ""name"":""Bobby+Sanabria+Conductin...",{}


In [4]:
people_df = people_df.drop_duplicates(subset=['id'], keep = 'first')

In [5]:
people_df

,object,id,ts,properties,linked_entities
0,person,145148,-1,"{""MBID"":null, ""name"":""Everything+Is+Illuminated""}",{}
1,person,297899,-1,"{""MBID"":null, ""name"":""Robin+O%27Brien""}",{}
2,person,250429,-1,"{""MBID"":null, ""name"":""Nicholas+Gunn++(2012)""}",{}
3,person,32765,-1,"{""MBID"":null, ""name"":""Aspasia+Stratigou""}",{}
4,person,18689,-1,"{""MBID"":null, ""name"":""Allison+Veltz""}",{}
...,...,...,...,...,...
595044,person,544215,-1,"{""MBID"":null, ""name"":""Sanaa+Kariakoo""}",{}
595045,person,298403,-1,"{""MBID"":null, ""name"":""Rock-a-teens""}",{}
595046,person,450896,-1,"{""MBID"":null, ""name"":""Jennifer+Lopez+Ft.+DJ+Mu...",{}
595047,person,53831,-1,"{""MBID"":null, ""name"":""Bobby+Sanabria+Conductin...",{}


In [6]:
def extract_person_name(row):
    json_lst = json.loads(row['properties'])
    person_id = row['id']

    new_rows_dict = {
                    'person_id' : person_id,
                    'person_name' : json_lst['name']
                   } 
    return new_rows_dict

In [7]:
%%time
new_persons_df = pd.DataFrame(list(people_df.apply(lambda row: extract_person_name(row), axis=1)))

CPU times: user 15 s, sys: 2.78 ms, total: 15 s
Wall time: 14.8 s


In [8]:
new_persons_df

,person_id,person_name
0,145148,Everything+Is+Illuminated
1,297899,Robin+O%27Brien
2,250429,Nicholas+Gunn++(2012)
3,32765,Aspasia+Stratigou
4,18689,Allison+Veltz
...,...,...
560922,544215,Sanaa+Kariakoo
560923,298403,Rock-a-teens
560924,450896,Jennifer+Lopez+Ft.+DJ+Mustard
560925,53831,Bobby+Sanabria+Conducting+The+Manhattan+School...


In [9]:
new_persons_df.to_pickle('data/new_persons_df.pickle')

### 7. Prepare train, validation and test datasets

In [4]:
df = pd.read_pickle('data/merged_df_datetime.pickle')

In [5]:
df.head()

,session_id,ts,user_id,track_id,person_id,date,time
0,287144,1390231051,44361,4698874,142266,2014-01-20,15:17:31
1,287144,1390231051,44361,838286,107103,2014-01-20,15:17:31
2,982046,1405452797,32894,838286,107103,2014-07-15,19:33:17
3,982049,1405702985,32894,838286,107103,2014-07-18,17:03:05
4,1873088,1406217037,23183,838286,107103,2014-07-24,15:50:37


In [6]:
df.shape

(31376602, 7)

In [7]:
users = df["user_id"].unique().tolist()
len(users)

45175

In [8]:
# 80% of user ids
users_train = [users[i] for i in range(round(0.8*len(users)))]

In [9]:
len(users_train)

36140

In [10]:
%%time
# split data into train and test set (80%-20% of users)
train_df = df[df['user_id'].isin(users_train)]
test_df = df[~df['user_id'].isin(users_train)]

CPU times: user 19.2 s, sys: 3.13 s, total: 22.4 s
Wall time: 19.1 s


In [11]:
train_df = train_df[['user_id', 'person_id', 'ts']].sort_values(by = ['user_id', 'ts'])
train_df['person_id'] = train_df['person_id'].astype(str)
train_grouped = train_df.groupby(by=['user_id'])['person_id'].apply(list).reset_index(name='persons_lst')

In [12]:
train_grouped.to_pickle('data/train_grouped.pickle')

In [13]:
%%time
train_left_out_df = shuffle(train_df)
left_out_df = train_left_out_df.drop_duplicates(subset=['user_id'])

CPU times: user 13.7 s, sys: 632 ms, total: 14.3 s
Wall time: 14.3 s


In [14]:
index1 = pd.MultiIndex.from_arrays([train_left_out_df[col] for col in ['user_id', 'person_id']])
index2 = pd.MultiIndex.from_arrays([left_out_df[col] for col in ['user_id', 'person_id']])

In [15]:
%%time
train_left_out_df = train_left_out_df.loc[~index1.isin(index2)]

CPU times: user 33.9 s, sys: 1.74 s, total: 35.7 s
Wall time: 36.3 s


In [16]:
train_left_out_df.shape

(28353219, 3)

In [17]:
train_left_out_df = train_left_out_df[['user_id', 'person_id', 'ts']].sort_values(by = ['user_id', 'ts'])

In [18]:
test_df = test_df[['user_id', 'person_id', 'ts']].sort_values(by = ['user_id', 'ts'])

In [19]:
train_left_out_df['person_id'] = train_left_out_df['person_id'].astype(str)
test_df['person_id'] = test_df['person_id'].astype(str)
left_out_df['person_id'] = left_out_df['person_id'].astype(str)

In [21]:
%%time
train_left_out_grouped = train_left_out_df.groupby(by=['user_id'])['person_id'].apply(list).reset_index(name='persons_lst')
test_grouped = test_df.groupby(by=['user_id'])['person_id'].apply(list).reset_index(name='persons_lst')

CPU times: user 9.94 s, sys: 666 ms, total: 10.6 s
Wall time: 10.7 s


In [22]:
train_left_out_grouped.to_pickle('data/train_left_out_grouped.pickle')
test_grouped.to_pickle('data/test_grouped.pickle')
left_out_df.to_pickle('data/left_out_df.pickle')